In [44]:
import numpy as np 
import pandas as pd
import matplotlib as plt
import math
import re
from collections import Counter

In [45]:
credits=pd.read_csv(r'F:\NLP project\tmdb_5000_credits.csv\tmdb_5000_credits.csv',encoding_errors = 'ignore')
movies=pd.read_csv(r'F:\NLP project\tmdb_5000_movies.csv\tmdb_5000_movies.csv',encoding_errors = 'ignore')


In [46]:
credits_column_renamed=credits.rename(columns={"movie_id":"id"})
movies_df_merged=movies.merge(credits_column_renamed,on='id')
movies_clean=movies_df_merged[['id','original_title','overview','label']]

In [47]:
col=movies_clean.columns
res=[]
for c in col:
    d=movies_clean[col].values.tolist()
    res.append(d)


In [48]:
def tf(word, count):
    return count[word] / sum(count.values())


def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)
 
 
def idf(word, count_list):
    return math.log(len(count_list) / (1 + n_containing(word, count_list)))


def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)


def as_num(x):
    y='{:.5f}'.format(x)
    return(y)


def euclideandistance(x,y):
    return np.sqrt(np.sum(np.square(x-y)))

In [49]:
stopwords = []
movie_tfidfvector=[]
corpus=[]
with open("stopword.txt", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        stopwords.append(line.strip("\n").strip())
countlist = []
for i in range(0,1470):
    with open(r"F:\NLP project\movie/"+str(i)+".txt", "r",encoding='utf-8') as f:    
        data = f.read()   
        
    text = re.sub(r"'s", " 's", data)
    text = re.sub(r"'ve", " 've", data)
    text = re.sub(r"n't", " n't", data)
    text = re.sub(r"'re", " 're", data)
    text = re.sub(r"'d", " 'd", data)
    text = re.sub(r"'ll", " 'll", data)
    text = re.sub(r"[^A-Za-z]", " ", data)
    do_lower_case=True
    if do_lower_case:
        text=text.strip().lower()
    tokens = text.split()
    text_words = [token for token in tokens if token not in stopwords]
    corpus.append(text_words)
    count = Counter(text_words)
    countlist.append(count)
    docvector=np.array([tfidf(word, count, countlist) for word in count])
    docvector1=np.reshape(docvector,(1,len(docvector)))
    docvector2 = abs(np.sort(-docvector1))
    movie_tfidfvector.append(docvector2[0][:5])


In [50]:
indices=pd.Series(movies_clean.index,index=movies_clean.original_title)

In [51]:
indices['Toy Story 3']

42

In [52]:
def recommend(a):
    vector=movie_tfidfvector[a]
    score=[]
    for i,j in enumerate(movie_tfidfvector):
        score.append([i,euclideandistance(j,vector)])
    Relatedmovie=sorted(score,key = lambda x: x[1],reverse = False)   
    res=Relatedmovie[1:11]
    movies_indices = [i[0] for i in res]
    series=movies_clean.label.iloc[movies_indices]
    return series.values

In [53]:
a=0
acclis=[]
for i in range(0,100):
    pre=0
    a=0
    for j in recommend(i):
        if j==movies_clean['label'][i]:
            a=a+1
    accuracy=a/10
    acclis.append(accuracy)


In [54]:
acc=sum(acclis)/len(acclis)
print('%.3f' % acc)

0.472
